In [8]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import glob
from astropy.table import Table
from weighted_kde import gaussian_kde as gss_kde
almost_black = '#262626'
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams.update({'font.size': 32, 
                     'axes.linewidth': 5,
                    'text.color': almost_black,
                    'xtick.major.size': 4,
                    'ytick.major.size': 4,
                    'legend.fancybox': True,
                    'figure.dpi': 300,
                    'legend.fontsize': 16,
                    'legend.framealpha': 0.8,
                    'legend.shadow': True,
                    'xtick.labelsize': 22,
                    'ytick.labelsize': 22})
import  bh_photo_z_validation  as pval
import yaml

import photoz_metrics_fn as pmf
reload(pmf)
import sml_mla as ml

from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from sklearn.preprocessing import Normalizer

In [2]:
#let's start with some dN/dz diagnotistics
tests = yaml.load(open('../validation/testConfig/WL_SAMPLE.yaml', 'r'))
z_tomo = tests['point']['bins']['MEAN_Z']
zbins = np.arange(0,2.0, 0.01)

In [ ]:
#get the effect from cosmic variance
cos_like_files = glob.glob('/Users/hoyleb/Sites/DES/PHOTOZ/MASKS/rotated_masks/cos_0alh_*.clean.fits')[0:5]
wide_field_like_files = glob.glob('/Users/hoyleb/Sites/DES/PHOTOZ/MASKS/rotated_masks/cos_2alh_*.clean.fits')

bse = ['MAG_AUTO_G', 'MAG_AUTO_I', 'MAG_AUTO_R', 'MAG_AUTO_Z']
feats = []
for i, fi in enumerate(bse):
    feats.append(fi)
    for j, fj in enumerate(bse):
        if j>i:
            feats.append(fi + '-' + fj)

simz = np.zeros((len(cos_like_files), len(z_tomo)-1,  len(zbins)-1)) 
knnz = np.zeros((len(cos_like_files), len(z_tomo)-1,  len(zbins)-1)) 
z_sim_truth = np.zeros((len(cos_like_files), len(z_tomo)-1, len(zbins))) - np.nan
for i in range(len(cos_like_files)):

    inputs, out_trn = ml.dataSet(cos_like_files[i], feats, ['Z']).loadData()
    nl = Normalizer()
    norm_feats = nl.fit_transform(inputs)
    
    clf = KNeighborsRegressor(n_neighbors=3, radius=2.0)
    clf.fit(norm_feats, out_trn['Z'])

    for jj in range(len(wide_field_like_files)):
        inputs, outputs = ml.dataSet(wide_field_like_files[jj], feats, ['Z']).loadData()
        ind_nn = np.isfinite(inputs[:, -1])
        for k in range(len(feats)-1):
            ind_nn *=  np.isfinite(inputs[:, k])
        inputs = inputs[ind_nn]
        outputs['Z'] = outputs['Z'][ind_nn]
        
        norm_feats1 = nl.transform(inputs)
        #neigh_id = clf.kneighbors(norm_feats1, n_neighbors=1, return_distance=False)
        #pred_z = out_trn['Z'][np.ravel(neigh_id)]
        pred_z = clf.predict(norm_feats1)

        photz = Table.read(wide_field_like_files[jj].replace('.fits', '.BPZ.fits'))
        photz = photz[ind_nn]
        for j in range(len(z_tomo)-1):
            ind = np.array(photz['MEAN_Z'] < z_tomo[j + 1]) * np.array(photz['MEAN_Z'] >= z_tomo[j])
            knnz[i, j] += np.histogram(pred_z[ind], bins=zbins)[0]
            simz[i, j] += np.histogram(outputs['Z'][ind], bins=zbins)[0]
    

In [ ]:
col = ['red', 'green', 'grey', 'yellow', 'purple']
for j in range(len(z_tomo)-1):
    f = plt.figure()
    for i in range(len(cos_like_files)-2):
        simz1 = simz[i, j]*1.0 / np.trapz(simz[i, j], zbins[0:-1] )
        knnz1 = knnz[i, j]*1.0 / np.trapz(knnz[i, j], zbins[0:-1] )
        plt.plot(zbins[0:-1], knnz1, '--', color=col[i], linewidth=3,alpha=0.5, label='Knn-z {:}'.format(i))
    plt.plot(zbins[0:-1], simz1, color=almost_black, linewidth=3, alpha=0.5, label='True-z {:}')
    plt.text(1, np.amax(simz1), 'Tomo-bin: ${:0.2f}<z<{:0.2f}$'.format(z_tomo[j],  z_tomo[j + 1]))
    
    plt.xlabel('Redshift')
    plt.ylabel('Density')
    plt.legend()